In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()   # choose kaggle.json from your computer


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"halimaakhter","key":"7c0444eea9c39abdfccda24b37cb8b84"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c spaceship-titanic
!unzip spaceship-titanic.zip -d spaceship_titanic


  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 503MB/s]
Archive:  spaceship-titanic.zip
  inflating: spaceship_titanic/sample_submission.csv  
  inflating: spaceship_titanic/test.csv  
  inflating: spaceship_titanic/train.csv  


In [ ]:
import pandas as pd

train = pd.read_csv("spaceship_titanic/train.csv")
test = pd.read_csv("spaceship_titanic/test.csv")
sample = pd.read_csv("spaceship_titanic/sample_submission.csv")

print(train.shape, test.shape, sample.shape)


(8693, 14) (4277, 13) (4277, 2)


Step 1 — Quick EDA & data audit (run these cells in Colab)

In [ ]:
import pandas as pd
import numpy as np

# 1) Peek
display(train.head(3))
display(test.head(3))

# 2) Basic schema
print("\nTRAIN INFO")
print(train.info())
print("\nTEST INFO")
print(test.info())

# 3) Target distribution (binary)
print("\nTransported distribution (train):")
print(train["Transported"].value_counts(normalize=True))

# 4) Column-by-column missingness & uniques
def audit(df, name):
    s = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "n_missing": df.isna().sum(),
        "missing_%": (df.isna().mean()*100).round(2),
        "n_unique": df.nunique()
    }).sort_values("missing_%", ascending=False)
    print(f"\n=== {name} audit ===")
    return s

display(audit(train, "train"))
display(audit(test, "test"))

# 5) Identify numeric/categorical/bool columns (will help later)
num_cols  = train.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols  = train.select_dtypes(include=["object","category"]).columns.tolist()
bool_cols = train.select_dtypes(include=["bool"]).columns.tolist()

# Remove target from feature lists if present
for lst in (num_cols, cat_cols, bool_cols):
    if "Transported" in lst:
        lst.remove("Transported")

print("\nnum_cols:", num_cols)
print("cat_cols:", cat_cols)
print("bool_cols:", bool_cols)

# 6) Quick leakage sanity check (same feature names except the target)
shared = set(train.columns) - {"Transported"}
print("\nFeature sets match train/test:", set(test.columns) == shared)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus



TRAIN INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None

TEST INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Co

,dtype,n_missing,missing_%,n_unique
CryoSleep,object,217,2.50,2
ShoppingMall,float64,208,2.39,1115
VIP,object,203,2.34,2
HomePlanet,object,201,2.31,3
Name,object,200,2.30,8473
Cabin,object,199,2.29,6560
VRDeck,float64,188,2.16,1306
Spa,float64,183,2.11,1327
FoodCourt,float64,183,2.11,1507
Destination,object,182,2.09,3



=== test audit ===


,dtype,n_missing,missing_%,n_unique
FoodCourt,float64,106,2.48,902
Spa,float64,101,2.36,833
Cabin,object,100,2.34,3265
ShoppingMall,float64,98,2.29,715
Name,object,94,2.20,4176
VIP,object,93,2.17,2
CryoSleep,object,93,2.17,2
Destination,object,92,2.15,3
Age,float64,91,2.13,79
HomePlanet,object,87,2.03,3



num_cols: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
cat_cols: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
bool_cols: []

Feature sets match train/test: True


Step 2 — Feature engineering + preprocessing pipeline

In [ ]:
# --- 2.1 Combine train/test to engineer features consistently ---
full = train.drop(columns=["Transported"]).copy()
full["__is_train__"] = True
test_copy = test.copy()
test_copy["__is_train__"] = False
both = pd.concat([full, test_copy], axis=0, ignore_index=True)


In [ ]:
# --- 2.2 Helper: add engineered features ---
import pandas as pd
import numpy as np

spend_cols = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # PassengerId: "gggg_pp" -> group id & number in group
    # Example: 0013_01  -> group_id=13, group_member=1
    gsplit = out["PassengerId"].str.split("_", expand=True)
    out["GroupID"] = gsplit[0].astype(int)
    out["GroupMember"] = gsplit[1].astype(int)

    # Cabin: "Deck/Num/Side" -> Deck (A–G/T), Num (int), Side (P/S)
    deck, num, side = (pd.Series(dtype="object"), pd.Series(dtype="float"), pd.Series(dtype="object"))
    if "Cabin" in out.columns:
        parts = out["Cabin"].str.split("/", expand=True)
        deck = parts[0]; num = pd.to_numeric(parts[1], errors="coerce"); side = parts[2]
    out["Deck"] = deck
    out["CabinNum"] = num
    out["Side"] = side

    # Spending features
    out["TotalSpent"] = out[spend_cols].fillna(0).sum(axis=1)
    out["NoSpend"] = (out["TotalSpent"] == 0)

    # Fill boolean columns to proper dtype (keep missing for imputer)
    for b in ["CryoSleep","VIP"]:
        if b in out.columns:
            out[b] = out[b].astype("boolean")  # pandas nullable boolean

    # Group-level features (need both train+test together)
    group_sizes = out.groupby("GroupID")["PassengerId"].transform("count")
    out["GroupSize"] = group_sizes
    out["IsAlone"] = (out["GroupSize"] == 1)

    # Simple text cleanup (optional)
    for c in ["HomePlanet","Destination","Deck","Side"]:
        if c in out.columns:
            out[c] = out[c].astype("category")

    return out

both_fe = add_features(both)

# Split back
train_fe = both_fe[both_fe["__is_train__"]].drop(columns="__is_train__").copy()
test_fe  = both_fe[~both_fe["__is_train__"]].drop(columns="__is_train__").copy()

print(train_fe.shape, test_fe.shape)
train_fe.head(3)


(8693, 22) (4277, 22)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,GroupID,GroupMember,Deck,CabinNum,Side,TotalSpent,NoSpend,GroupSize,IsAlone
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,Maham Ofracculy,1,1,B,0.0,P,0.0,True,1,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,Juanna Vines,2,1,F,0.0,S,736.0,False,1,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,Altark Susent,3,1,A,0.0,S,10383.0,False,2,False


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# 1) Choose columns explicitly
num_cols = [
    "Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck",
    "CabinNum","GroupMember","TotalSpent","GroupSize"
]
cat_cols = [
    "HomePlanet","CryoSleep","Destination","VIP","Deck","Side",
    "NoSpend","IsAlone","GroupID"   # treat GroupID as categorical
]

# 2) Build X, y (drop raw identifiers)
drop_cols = ["PassengerId","Name","Cabin"]
X = train_fe.drop(columns=drop_cols ).copy()
y = train["Transported"].astype(int)
X_test = test_fe.drop(columns=drop_cols).copy()



# 3) Ensure dtypes
for c in num_cols:
    if c in X.columns:
        X[c] = pd.to_numeric(X[c], errors="coerce")
        X_test[c] = pd.to_numeric(X_test[c], errors="coerce")
for c in cat_cols:
    if c in X.columns:
        X[c] = X[c].astype("object")
        X_test[c] = X_test[c].astype("object")

# 4) Preprocessors
numeric_pre = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))
])
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

categorical_pre = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe)
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pre, [c for c in num_cols if c in X.columns]),
        ("cat", categorical_pre, [c for c in cat_cols if c in X.columns]),
    ]
)

print("final numeric:", [c for c in num_cols if c in X.columns])
print("final categorical:", [c for c in cat_cols if c in X.columns])


final numeric: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum', 'GroupMember', 'TotalSpent', 'GroupSize']
final categorical: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side', 'NoSpend', 'IsAlone', 'GroupID']


In [ ]:
X [ [c for c in num_cols if c in X.columns]].head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinNum,GroupMember,TotalSpent,GroupSize
0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1
1,24.0,109.0,9.0,25.0,549.0,44.0,0.0,1,736.0,1
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0.0,1,10383.0,2
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0.0,2,5176.0,2
4,16.0,303.0,70.0,151.0,565.0,2.0,1.0,1,1091.0,1


In [ ]:
X [ [c for c in num_cols if c in X.columns]].head()
X[ [c for c in cat_cols if c in X.columns]].head()

,HomePlanet,CryoSleep,Destination,VIP,Deck,Side,NoSpend,IsAlone,GroupID
0,Europa,False,TRAPPIST-1e,False,B,P,True,True,1
1,Earth,False,TRAPPIST-1e,False,F,S,False,True,2
2,Europa,False,TRAPPIST-1e,True,A,S,False,False,3
3,Europa,False,TRAPPIST-1e,False,A,S,False,False,3
4,Earth,False,TRAPPIST-1e,False,F,S,False,True,4


In [ ]:
cat_cols


['HomePlanet',
 'CryoSleep',
 'Destination',
 'VIP',
 'Deck',
 'Side',
 'NoSpend',
 'IsAlone',
 'GroupID']

In [ ]:
num_cols

['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'CabinNum',
 'GroupMember',
 'TotalSpent',
 'GroupSize']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print(X [ [c for c in num_cols if c in X.columns]].head())
print(X[ [c for c in cat_cols if c in X.columns]].head())
X = X.replace({pd.NA: np.nan})
X_test = X_test.replace({pd.NA: np.nan})

# 3.1 hold-out split for a quick honest check
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(X_tr.head())

# 3.2 model = preprocessing pipeline + logistic regression
logreg = Pipeline(steps=[
    ("prep", preprocess),
    # 'saga' and 'liblinear' both work with sparse matrices; use saga for robustness
    ("clf", LogisticRegression(max_iter=2000, solver="saga"))
])

# 3.3 train & validate
logreg.fit(X_tr, y_tr)
val_pred = logreg.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
print(f"Validation accuracy: {val_acc:.4f}")

# Optional detailed report (comment out if too verbose)
print(classification_report(y_val, val_pred, digits=3))


    Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  CabinNum  \
0  39.0          0.0        0.0           0.0     0.0     0.0       0.0   
1  24.0        109.0        9.0          25.0   549.0    44.0       0.0   
2  58.0         43.0     3576.0           0.0  6715.0    49.0       0.0   
3  33.0          0.0     1283.0         371.0  3329.0   193.0       0.0   
4  16.0        303.0       70.0         151.0   565.0     2.0       1.0   

   GroupMember  TotalSpent  GroupSize  
0            1         0.0          1  
1            1       736.0          1  
2            1     10383.0          2  
3            2      5176.0          2  
4            1      1091.0          1  
  HomePlanet CryoSleep  Destination    VIP Deck Side NoSpend IsAlone GroupID
0     Europa     False  TRAPPIST-1e  False    B    P    True    True       1
1      Earth     False  TRAPPIST-1e  False    F    S   False    True       2
2     Europa     False  TRAPPIST-1e   True    A    S   False   False       3
3

/tmp/ipython-input-1051441657.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({pd.NA: np.nan})
/tmp/ipython-input-1051441657.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.replace({pd.NA: np.nan})


Validation accuracy: 0.7884
              precision    recall  f1-score   support

           0      0.787     0.787     0.787       863
           1      0.790     0.790     0.790       876

    accuracy                          0.788      1739
   macro avg      0.788     0.788     0.788      1739
weighted avg      0.788     0.788     0.788      1739



In [ ]:
# 3.4 fit on ALL training data and predict the test set
logreg.fit(X, y)

# predict_proba -> threshold at 0.5 to produce True/False
proba = logreg.predict_proba(X_test)[:, 1]
y_test_pred = proba >= 0.5
print(y_test_pred[:5], y_test_pred.dtype)  # sanity check


[ True False  True  True  True] bool


In [ ]:
# 3.5 create submission.csv in the exact Kaggle format
submission = pd.DataFrame({
    "PassengerId": test_fe["PassengerId"],
    "Transported": y_test_pred.astype(bool)  # ensure True/False
})
submission.to_csv("submission.csv", index=False)
submission.head()


,PassengerId,Transported
8693,0013_01,True
8694,0018_01,False
8695,0019_01,True
8696,0021_01,True
8697,0023_01,True


In [ ]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install xgboost


In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

xgb_model = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", XGBClassifier(
        n_estimators=300,        # number of boosting rounds
        learning_rate=0.05,      # smaller lr → smoother learning
        max_depth=5,             # tree depth
        subsample=0.8,           # row sampling
        colsample_bytree=0.8,    # feature sampling
        eval_metric="logloss",   # required to suppress warnings
        use_label_encoder=False,
        random_state=42
    ))
])

xgb_model.fit(X_tr, y_tr)
val_pred = xgb_model.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
print(f"XGBoost validation accuracy: {val_acc:.4f}")

xgb_model.fit(X, y)
test_pred = xgb_model.predict(X_test)

submission_xgb = pd.DataFrame({
    "PassengerId": test_fe["PassengerId"],
    "Transported": test_pred.astype(bool)
})
submission_xgb.to_csv("submission_xgb.csv", index=False)
submission_xgb.head()


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [03:03:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost validation accuracy: 0.8079


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [03:03:54] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,PassengerId,Transported
8693,0013_01,True
8694,0018_01,False
8695,0019_01,True
8696,0021_01,True
8697,0023_01,True


In [ ]:
from google.colab import files
files.download("submission_xgb.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>